<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/6-decision-trees/decision_trees_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Decision Trees Exercises

Decision Trees are versatile Machine Learning algorithms that can perform
both classification and regression tasks, and even multioutput tasks. They are
powerful algorithms, capable of fitting complex datasets.

Decision Trees are also the fundamental components of Random Forests, which are among the most powerful Machine Learning algorithms available today.



##Setup

In [17]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

from sklearn.datasets import load_iris
from sklearn.datasets import make_moons

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

from sklearn.base import clone
from scipy.stats import mode

from sklearn.metrics import accuracy_score

from sklearn.tree import export_graphviz

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from graphviz import Source
from matplotlib.colors import ListedColormap

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

##Exercise-1: Train and fine-tune a Decision Tree

Train and fine-tune a Decision Tree for the moons dataset by following these
steps:

1. Use `make_moons(n_samples=10000, noise=0.4)` to generate a moons dataset.
2. Use `train_test_split()` to split the dataset into a training set and a test set.
3. Use grid search with `cross-validation` (with the help of the `GridSearchCV` class) to find good hyperparameter values for a `DecisionTreeClassifier`. Hint: try various values for `max_leaf_nodes`.
4. Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to 87%
accuracy.



###Step-1: Generate a moons dataset 

Let's generate a moons dataset using `make_moons(n_samples=10000, noise=0.4)`.

Adding `random_state=42` to make this notebook's output constant:

In [2]:
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

###Step-2: Split dataset

Let's split it into a training set and a test set using `train_test_split()`.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###Step-3: Search hyperparameter using grid search

Let's use grid search with cross-validation (with the help of the `GridSearchCV` class) to find good hyperparameter values for a `DecisionTreeClassifier`. 

Hint: try various values for `max_leaf_nodes`.

In [6]:
params = {"max_leaf_nodes": list(range(2, 100)), "min_samples_split": [2, 3, 4]}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(x_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [7]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

###Step-4: Running full training using best hyperparameters

Let's train it on the full training set using these hyperparameters, and measure your model's performance on the test set. 

You should get roughly 85% to 87% accuracy.

By default, `GridSearchCV` trains the best model found on the whole training set (you can change this by setting `refit=False`), so we don't need to do it again. 

We can simply evaluate the model's accuracy:

In [10]:
y_pred = grid_search_cv.predict(x_test)
accuracy_score(y_test, y_pred)

0.8695

##Exercise-2: Implement Random forest fundamental

Let's grow a forest by following these steps:

1. Generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use Scikit-
Learn’s `ShuffleSplit` class for this.
2. Train one Decision Tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 Decision Trees on the test
set. Since they were trained on smaller sets, these Decision Trees will likely
perform worse than the first Decision Tree, achieving only about 80%
accuracy.
3. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s `mode()` function for this). This approach gives you majority-vote predictions over the test set.
4. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). 

Congratulations, you have trained a Random Forest classifier!

###Step-1: Generate dataset

Let's generate 1,000 subsets of the training set, each containing 100 instances selected randomly. 

Hint: you can use Scikit-Learn's `ShuffleSplit` class for this.

In [13]:
n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(x_train) - n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(x_train):
  x_mini_train = x_train[mini_train_index]
  y_mini_train = y_train[mini_train_index]
  mini_sets.append((x_mini_train, y_mini_train))

###Step-2: Train one Decision Tree

Let's train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. 

Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [15]:
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (x_mini_train, y_mini_train) in zip(forest, mini_sets):
  tree.fit(x_mini_train, y_mini_train)

  y_pred = tree.predict(x_test)
  accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

###Step-3: Make prediction on 1,000 Decision Trees

Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy's `mode()` function for this). 

This gives you `_majority-vote predictions_` over the test set.

In [16]:
y_pred = np.empty([n_trees, len(x_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
  y_pred[tree_index] = tree.predict(x_test)

In [18]:
y_pred_majority_votes, n_votes = mode(y_pred, axis=0)

###Step-4: Evaluate these predictions

Let's evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher).

In [19]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872

Congratulations, you have trained a Random Forest classifier!